In [ ]:
import numpy as np
import pandas as pd
from datetime import timedelta
import pickle
import os
import warnings
from IPython.display import display

warnings.filterwarnings("ignore")

#algorithm list selection
algorithms=[]
for filename in os.listdir('../input/weather-forecasting-at-ria-arousa-spain-using-ai/'):
    if filename.endswith('.al'):
        algorithms.append(filename)
    
algorithms.sort()  

print("Select an algorithm number\n")
for n,alg in zip(np.arange(0,len(algorithms)),algorithms):
    print(n,alg)
print("\n")    
alg_number=int(input())

#load algorithm
algo=pickle.load(open("../input/weather-forecasting-at-ria-arousa-spain-using-ai/"+algorithms[alg_number], 'rb'))

 
#defining url
today=pd.to_datetime("today")
head1="http://mandeo.meteogalicia.es/thredds/ncss/modelos/WRF_HIST/"+algo["mod_res"]
head2=today.strftime("/%Y/%m/wrf_arw_det_history_")+algo["mod_res"]
head3=today.strftime("_%Y%m%d_0000.nc4?")
head=head1+head2+head3
 
var1="var=dir&var=mod&var=wind_gust&var=mslp&var=temp&var=rh&var=visibility&var=lhflx"
var2="&var=lwflx&var=conv_prec&var=prec&var=swflx&var=shflx&var=cape&var=cin&var=cfh"
var3="&var=cfl&var=cfm&var=cft&var=HGT500&var=HGT850&var=T500&var=T850"
var=var1+var2+var3
 
f_day=(today+timedelta(days=2)).strftime("%Y-%m-%d") 
tail="&time_start="+today.strftime("%Y-%m-%d")+"T01%3A00%3A00Z&time_end="+f_day+"T23%3A00%3A00Z&accept=csv"
 
 
dffinal=pd.DataFrame() 
for coor in list(zip(algo["lat"],algo["lon"],np.arange(0,len(algo["lat"])).astype(str))):
  dffinal=pd.concat([dffinal,pd.read_csv(head+var+"&latitude="+str(coor[0])+"&longitude="+str(coor[1])+tail,).add_suffix(str(coor[2]))],axis=1)
 
#filter all columns with lat lon and date
dffinal=dffinal.filter(regex='^(?!(lat|lon|date).*?)')
 
#remove column string between brakets
new_col=[c.split("[")[0]+c.split("]")[-1] for c in dffinal.columns]
for col in zip(dffinal.columns,new_col):
  dffinal=dffinal.rename(columns = {col[0]:col[1]})
 
dffinal=dffinal.set_index(pd.date_range(start=today.strftime("%Y-%m-%d"), end=(today+timedelta(days=3)).strftime("%Y-%m-%d"), freq="H")[1:-1]) 

#Machine learning and meteorological model probabilistic result 



#select x variable from the model
df_x=dffinal[algo["x_var"]]

#select D interval
if algo["D"]==0:
  df_x= df_x.iloc[0:23]
if algo["D"]==1:
  df_x=df_x.iloc[23:47]
if algo["D"]==2:
  df_x =df_x.iloc[47:72]

#machine learning probabilistic result
ml_prob= algo["model"].predict_proba(algo["pca"].transform(algo["scaler"].transform(df_x))) 
#ml_result=pd.DataFrame(ml_prob,columns=algo["model"].classes_).set_index(df_x.index).applymap("{:.0%}".format).add_suffix('_ml')
ml_result=pd.DataFrame(ml_prob,columns=algo["model"].classes_).set_index(df_x.index).add_suffix('_ml')
ml_result["max_ml"]=ml_result.idxmax(axis=1)
ml_result.iloc[:,0:-1]=ml_result.iloc[:,0:-1].applymap(lambda n: '{:.0%}'.format(n))
#display("Machine learning results",ml_result)
#display("Machine learning Classification report",algo["class"])
display(ml_result,algo["class"])
#meteorological model results if they exists
if algo['x_and_y_same']:
  #meteorological variable at point 0 nearest point
  met_var_p0=list(algo['met_var_sc'].keys())[0]

  #meteorological score columns
  #met_columns=algo["met_var_sc"][met_var_p0][1].applymap("{:.0%}".format)
  met_columns=algo["met_var_sc"][met_var_p0][1]
  #label meteorological model variables
  
  df_x[met_var_p0+"_l"]=pd.cut(df_x[met_var_p0],bins = algo["interval"],)
  df_x[met_var_p0+"_l"]=df_x[met_var_p0+"_l"].map({a:b for a,b in zip(algo["interval"],algo["labels"])})
  #get meteorological probability result
  array_p=[]
  for i in range(0,len(df_x)):
    array_p.append(met_columns[df_x[met_var_p0+"_l"][i]].values)
  met_result=pd.DataFrame(array_p,columns=met_columns.index,index=df_x.index).add_suffix('_met')
  met_result["max_met"]=met_result.iloc[:,:-1].idxmax(axis=1)
  met_result=met_result.drop(["entropy_met"],axis=1)
  met_result.iloc[:,0:-1]=met_result.iloc[:,0:-1].applymap(lambda n: '{:.0%}'.format(n))
  display(met_result,algo['met_var_sc'][met_var_p0][3])  
else:
  print("\n","No meteorological model results")


#@title Save results
#save results

#library required
! pip install openpyxl

#save results
import openpyxl

out_file=today.strftime("%d%m%Y_")+algorithms[alg_number]+".xlsx"

#apply % format to machine learning result
algo["class"].iloc[:,0:3]=algo["class"].iloc[:,0:3].applymap("{:.0%}".format)
algo["class"].iloc[-3,3]="{:.0%}".format(algo["class"].iloc[-3,3])

#cross validation
cros=algo['cros_val']
df_cros=pd.DataFrame({"test_accuracy":cros["test_accuracy"],'test_f1_macro':cros['test_f1_macro'],
                      'test_f1_weighted':cros['test_f1_weighted']}).applymap("{:.1%}".format)

if algo['x_and_y_same']:
  with pd.ExcelWriter(out_file) as writer:
    #save results  
    pd.concat([met_result,ml_result],axis=1).to_excel(writer, sheet_name='results')

    #save precision
    ml_precision=algo["column"].add_suffix('_ml').applymap("{:.0%}".format)
    metmodel_precision=algo['met_var_sc'][met_var_p0][1].add_suffix("_metmodel").applymap("{:.0%}".format)
    precision_save=pd.concat([ml_precision,metmodel_precision],axis=1)
    columns_sorted=precision_save.columns.sort_values()
    precision_save[columns_sorted].to_excel(writer,sheet_name="precision")
    
    #save classification report
    algo['met_var_sc'][met_var_p0][3].iloc[:,0:3]=algo['met_var_sc'][met_var_p0][3].iloc[:,0:3].applymap("{:.0%}".format)
    algo['met_var_sc'][met_var_p0][3].iloc[-3,3]="{:.0%}".format(algo['met_var_sc'][met_var_p0][3].iloc[-3,3])
    report_save=pd.concat([algo['met_var_sc'][met_var_p0][3].add_suffix("_metmodel"),algo["class"].add_suffix('_ml')],axis=1)
    columns_sorted=report_save.columns.sort_values()
    report_save[columns_sorted].to_excel(writer,sheet_name="class_report",)

    #save cross validation               
    df_cros.to_excel(writer,sheet_name="cros_validation") 

else:
  with pd.ExcelWriter(out_file) as writer:  
    ml_result.to_excel(writer, sheet_name='ml_model')
    algo["column"].applymap("{:.0%}".format).to_excel(writer,sheet_name="ml_precision")
    algo["class"].to_excel(writer,sheet_name="ml_clas_report")
    df_cros.to_excel(writer,sheet_name="cros_validation")